In [1]:
import os
import pandas as pd 
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSCanonical, PLSRegression, CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, cross_val_score
import matplotlib  as mpl
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
%matplotlib inline
import os, shutil, glob
from PIL import Image
from itertools import cycle
from random import randint
import re, math
import seaborn as sns; sns.set_style("white")
from sklearn.manifold import TSNE
import umap.umap_ as umap
import datetime
import gc
from pathlib import Path

os.getcwd()

'/home/jovyan/maris/2022/Amelie_TX'

### Read data

In [2]:
BaseDir = '/home/jovyan/cpp_work/results/'
# sorted(os.listdir(BaseDir))

### Input directories

In [3]:
InputFolders = []
NameContains = input("Enter part of name(s) of the folders: ") # e.g.: TX-.*GR-v2
for f in (f for f in os.listdir(BaseDir) if re.match(NameContains, f)):
    InputFolders.append(f)
InputFolders.sort()
print ('InputFolders: ' + str(InputFolders))

Enter part of name(s) of the folders:  TX-.*GR-v2


InputFolders: ['TX-Plate1-MDA-MB-231-48h-L01-GR-v2', 'TX-Plate2-MDA-MB-231-48h-L02-GR-v2']


### Output directory

In [4]:
OutputDir = input("Enter name for output folder: ") # e.g.: ImageMeanFeatures
if not os.path.exists(OutputDir): 
    os.makedirs(OutputDir)

Enter name for output folder:  ImageMedianFeatures


In [5]:
now = datetime.datetime.now()
print ('Current date and time : ')
print (now.strftime('%Y-%m-%d %H:%M:%S'))

Current date and time : 
2022-04-06 12:03:24


In [6]:
groupedbyImageAllPlates = pd.DataFrame()

for oneplate in InputFolders[:]:

        DataFrameDictionary = {}
        ObjectList = ['featICF_nuclei', 'featICF_cells', 'featICF_cytoplasm']
        for obj in ObjectList:
            DataFrameDictionary[obj] = pd.DataFrame()
            ReadingFile = 0
            #for InpF in InputFolders:
            DataFromFolder =  pd.DataFrame()
            feature_files = glob.glob(BaseDir + '/*' + oneplate + '*/*/*/' + obj + '.csv', recursive=True)
          
            
            
            for file in feature_files[-1:]:
                print(file)
                DataFromOneFile =  pd.read_csv(file, sep=",", low_memory=False)
                print(DataFromOneFile.shape)
                ReadingFile += 1    
                DataFromFolder = DataFromFolder.append(DataFromOneFile.iloc[:,:], ignore_index=True)
                print(DataFromFolder.shape)

                DataFrameDictionary[obj] = DataFrameDictionary[obj].append(DataFromFolder.iloc[:,:], ignore_index=True)
            DataFrameDictionary[obj].columns = [str(col) + '_' + re.sub('_.*', '', re.sub('featICF_', '', obj)) for col in DataFrameDictionary[obj]]
            print('Found %s files' %(ReadingFile))
            print ('Dataframe contains {} columns and {} rows.'.format(DataFrameDictionary[obj].shape[1],
                                                                  DataFrameDictionary[obj].shape[0]))

        df = DataFrameDictionary['featICF_nuclei'].merge(DataFrameDictionary['featICF_cells'], left_on = [ 'Metadata_Barcode_nuclei',
        'Metadata_Site_nuclei', 'Metadata_Well_nuclei','Parent_cells_nuclei'],
                           right_on = [ 'Metadata_Barcode_cells',
        'Metadata_Site_cells', 'Metadata_Well_cells','ObjectNumber_cells'], how='left')
        print(df.shape)

        df = df.merge(DataFrameDictionary['featICF_cytoplasm'], left_on = [ 'Metadata_Barcode_nuclei',
        'Metadata_Site_nuclei', 'Metadata_Well_nuclei','Parent_cells_nuclei'],
                           right_on = [ 'Metadata_Barcode_cytoplasm',
        'Metadata_Site_cytoplasm', 'Metadata_Well_cytoplasm','ObjectNumber_cytoplasm'], how='left')
        print(df.shape)
        df.dropna(inplace=True)
        df.reset_index(drop=True, inplace=True)
        print(df.shape)
        print('Input: {}'.format(NameContains))
        print('Output: {}'.format(OutputDir))
        print('Dataset shape: {}'.format(df.shape))
        
        del DataFrameDictionary
        gc.collect()
        
        NrOfObjects = df.shape[0]

        Wells = sorted(list(set(df['Metadata_Well_nuclei'])))
        NrOfWells = len(Wells)
        Rows = sorted(list(set([w[0] for w in Wells])))
        print(*Rows)
        NrOfRows = len(Rows)
        Columns = sorted(list(set([w[1:] for w in Wells])))
        NrOfColumns = len(Columns)
        print(*Columns)
        Sites = sorted(list(set(df['Metadata_Site_nuclei'])))
        NrOfSites = len(Sites)
        # print('Plate complete: {}'.format(NrOfRows*NrOfColumns==NrOfWells))
        print('Number of sites: {}'.format(NrOfSites))
        AllWells =[]
        for R in Rows:
            for C in Columns:
                RC = R + C
                # print(RC)
                AllWells += [RC]
        # print(len(AllWells))
        print('Number of wells: {} (full plate is {})'.format(NrOfWells, len(AllWells)))
        
        df['ImageID'] =  df['Metadata_Barcode_nuclei'] + '_' + df['Metadata_Well_nuclei'] + '_' + df['Metadata_Site_nuclei'].astype(str)

        numeric_columns = list()
        for a in df.columns:
            if (df.dtypes[a] == 'float64') | (df.dtypes[a] == 'int64') :
                numeric_columns.append(a)
        print('Found {} numeric columns'.format(len(numeric_columns)))

        dictOfnumeric = { i : np.nanmedian for i in numeric_columns}

        groupedbyImage = df.groupby('ImageID').agg(dictOfnumeric)
        groupedbyImage.to_csv('{}/ImageMediansPlate{}.csv'.format(OutputDir, oneplate), sep=';')
        frames = [groupedbyImageAllPlates, groupedbyImage]
        groupedbyImageAllPlates = pd.concat(frames)
        print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
        print('Now the shape of image dataset is: {}'.format(groupedbyImageAllPlates.shape))
        print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
        del df
        del groupedbyImage
        gc.collect()
        
groupedbyImageAllPlates.to_csv('{}/ImageMediansAllPlates.csv'.format(OutputDir), sep=';')

/home/jovyan/cpp_work/results/TX-Plate1-MDA-MB-231-48h-L01-GR-v2/1318/863/featICF_nuclei.csv
(229074, 733)
(229074, 733)
Found 1 files
Dataframe contains 733 columns and 229074 rows.
/home/jovyan/cpp_work/results/TX-Plate1-MDA-MB-231-48h-L01-GR-v2/1318/863/featICF_cells.csv
(175735, 734)
(175735, 734)
Found 1 files
Dataframe contains 734 columns and 175735 rows.
/home/jovyan/cpp_work/results/TX-Plate1-MDA-MB-231-48h-L01-GR-v2/1318/863/featICF_cytoplasm.csv
(175735, 725)
(175735, 725)
Found 1 files
Dataframe contains 725 columns and 175735 rows.
(229074, 1467)
(229074, 2192)
(199315, 2192)
Input: TX-.*GR-v2
Output: ImageMedianFeatures
Dataset shape: (199315, 2192)
A B C D E F G H I J K L M N O P
01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
Number of sites: 4
Number of wells: 384 (full plate is 384)
Found 2126 numeric columns
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Now the shape of image dataset is: (1532, 2126)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
/home/jov

In [7]:
now = datetime.datetime.now()
print ('Current date and time : ')
print (now.strftime('%Y-%m-%d %H:%M:%S'))

Current date and time : 
2022-04-06 12:54:30
